In [1]:
import os
import sys
import dotenv
import pandas as pd
sys.path.extend(['..', '../..'])

In [2]:
import nature_go_client

dotenv.load_dotenv()
NATURE_GO_USERNAME = os.getenv("NATURE_GO_USERNAME")
NATURE_GO_PASSWORD = os.getenv("NATURE_GO_PASSWORD")
client = nature_go_client.NatureGoClient(username=NATURE_GO_USERNAME, password=NATURE_GO_PASSWORD)
client.login()

## Retrieve species with missing text content through Nature go API

In [3]:
species_list = pd.DataFrame(client.get_labeled_species(limit=5, multiplechoicequestions=False))
species_list = species_list.sort_values(by='num_observations_total', ascending=False)
species_list

http://nature-go.edouardleurent.com/api/species/labeled/?limit=5&offset=0&multiplechoicequestions=False


,id,type,scientificNameWithoutAuthor,commonNames,genus,family,gbif_id,powo_id,wikipedia_word_count,number_of_occurrences,...,illustration,illustration_transparent,illustration_reference,illustration_reference_transparent,reference_image_url,rarity,illustration_url,display_name,num_observations_total,num_questions_total
0,182,plant,Euphorbia amygdaloides,"[Wood Spurge, Laptele câinelui (RO), Greater H...",Euphorbia,Euphorbiaceae,3065239.0,345568-1,155,2155,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Common,http://nature-go.edouardleurent.com/media/spec...,Wood Spurge,1,0
1,3120,plant,Narcissus bicolor,"[Two-color daffodil, Two-coloured Daffodil]",Narcissus,Amaryllidaceae,2858359.0,65928-1,-1,28,...,None,None,None,None,,Rare,http://nature-go.edouardleurent.com/static/img...,Two-color daffodil,1,0
2,4681,plant,Papaver cambricum,"[Welsh Poppy, Poppy]",Papaver,Papaveraceae,3602683.0,673433-1,495,10,...,None,None,None,None,,Rare,http://nature-go.edouardleurent.com/static/img...,Welsh Poppy,1,0
3,10752,plant,Stellaria palustris,"[Meadow starwort, Marsh Stitchwort, Marsh star...",Stellaria,Caryophyllaceae,8127028.0,329693-2,32,1,...,None,None,None,None,,Rare,http://nature-go.edouardleurent.com/static/img...,Meadow starwort,1,0
4,32982,plant,Scilla hyacinthoides,"[Hyacinth bluebell, Hyacinth squill, Scilla]",Scilla,Asparagaceae,2767320.0,540833-1,139,-1,...,None,None,None,None,,Legendary,http://nature-go.edouardleurent.com/static/img...,Hyacinth bluebell,1,0


In [4]:
species = species_list.iloc[0]
print(species)

id                                                                                  182
type                                                                              plant
scientificNameWithoutAuthor                                      Euphorbia amygdaloides
commonNames                           [Wood Spurge, Laptele câinelui (RO), Greater H...
genus                                                                         Euphorbia
family                                                                    Euphorbiaceae
gbif_id                                                                       3065239.0
powo_id                                                                        345568-1
wikipedia_word_count                                                                155
number_of_occurrences                                                              2155
occurences_cdf                                                                 0.581424
rarity_gpt                      

## Run generation

In [5]:
from backend.nature_go.generation import gemini
gemini.configure()

In [6]:
from backend.nature_go.generation import description_generation

summary, response = description_generation.generate_descriptions(
    generate_text=gemini.generate_text,
    species=species,
)
print(f'{len(summary)} parts generated.', '\n')
for paragraph in summary:
    print(paragraph, '\n')

3 parts generated. 

Today I encountered a most curious plant, growing in the dappled shade of a beechwood. It's called *Euphorbia amygdaloides*, or Wood Spurge, and is a rather unassuming fellow at first glance. A bushy green fellow, not much taller than my knee, with leaves that are as soft as a young lamb's fleece. I've been told that the plant spreads like wildfire, but I've yet to see any sign of its invasive nature in this quiet corner of the forest. 

Now, the real surprise came when I took a closer look at its flowers.  They are not your typical rose or lily, oh no, these are something altogether different!  It seems the plant plays a trick on us, with a complex 'cyathium' that looks like a single flower but is actually a cluster of tiny blooms.  And they're not just any tiny blooms, mind you, but a vibrant green-yellow colour, like the most brilliant sunshine you've ever seen!  

One final thing I must mention. Though it is a beautiful sight, there's a certain danger to this p

In [7]:
from backend.nature_go.generation import question_generation

material = '\n '.join(summary)
questions, _ = question_generation.generate_questions(
    generate_text=gemini.generate_text,
    species=species,
    material=material,
)
questions

[{'question': 'What is the typical height of Wood Spurge?',
  'choices': ['It can grow very tall.',
   "It's usually about knee-high.",
   "It's a creeping groundcover.",
   'It can reach the height of a tree.'],
  'correct_choice': 1},
 {'question': "What makes Wood Spurge's flowers unique?",
  'choices': ['They resemble typical roses.',
   'They are bright red.',
   'They are actually clusters of tiny blooms.',
   'They have a strong, sweet smell.'],
  'correct_choice': 2},
 {'question': 'What is the danger associated with Wood Spurge?',
  'choices': ['It attracts harmful insects.',
   'Its leaves are sharp and can cause cuts.',
   'Its sap is poisonous.',
   "It's highly flammable."],
  'correct_choice': 2}]